In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
%cd drive/MyDrive/Neuromatch

/content/drive/MyDrive/Neuromatch


In [3]:
!pip install ipython-autotime
!pip install -q swifter
%load_ext autotime

time: 328 µs (started: 2023-07-26 11:42:38 +00:00)


In [4]:
import numpy as np
import pandas as pd

time: 322 ms (started: 2023-07-26 11:42:38 +00:00)


In [5]:
true_data = pd.read_csv('datasets/True.csv')
fake_data = pd.read_csv('datasets/Fake.csv')

time: 2.08 s (started: 2023-07-26 11:42:38 +00:00)


In [6]:
true_data["label"] = np.ones(len(true_data),dtype=int)
fake_data["label"] = np.zeros(len(fake_data),dtype=int)

time: 2.31 ms (started: 2023-07-26 11:42:40 +00:00)


In [7]:
data = pd.concat((true_data,fake_data),axis=0)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44898 entries, 0 to 23480
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.1+ MB
None
time: 101 ms (started: 2023-07-26 11:42:40 +00:00)


In [8]:
data = data.sample(frac=1)
data.head(10)

,title,text,subject,date,label
21137,GA TOWN MANDATES GUN OWNERSHIP: Here’s What Ha...,The results of this town s gun ownership manda...,left-news,"Jan 11, 2016",0
19299,Turkey wants Iraq's Kurdish region to drop ref...,ANKARA (Reuters) - The planned independence re...,worldnews,"September 22, 2017",1
14236,Mnangagwa will be sworn in as Zimbabwe preside...,HARARE (Reuters) - Zimbabwe s former vice pres...,worldnews,"November 21, 2017",1
8619,WATCH: The Media Gives MI’s Gov. Rick Snyder ...,Michigan s governor Rick Snyder is desperately...,News,"January 21, 2016",0
18147,WHERE WAS MEDIA OUTRAGE After This WOMAN DELIB...,The media has been giving wall-to-wall coverag...,left-news,"Aug 13, 2017",0
5270,White House bars some news organizations from ...,WASHINGTON (Reuters) - The White House exclude...,politicsNews,"February 24, 2017",1
7813,Clinton leads by 7 points as Trump faces grope...,NEW YORK (Reuters) - Democratic presidential c...,politicsNews,"October 14, 2016",1
10107,THE LEFT LOSES AGAIN: Third Quarter Economic E...,The news that the forecast for the third quart...,politics,"Aug 19, 2017",0
17534,Factbox: U.S. congressional leaders on Iran nu...,WASHINGTON (Reuters) - U.S. President Donald T...,worldnews,"October 13, 2017",1
15786,(VIDEO) HILLARY CLINTON: RELIGIOUS BELIEFS MUS...,,politics,"Apr 25, 2015",0


time: 53.2 ms (started: 2023-07-26 11:42:41 +00:00)


In [9]:
data = data.drop("date",axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44898 entries, 21137 to 17594
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   label    44898 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.7+ MB
time: 105 ms (started: 2023-07-26 11:42:41 +00:00)


In [ ]:
import nltk
nltk.download('all')

In [11]:
data.head()

,title,text,subject,label
21137,GA TOWN MANDATES GUN OWNERSHIP: Here’s What Ha...,The results of this town s gun ownership manda...,left-news,0
19299,Turkey wants Iraq's Kurdish region to drop ref...,ANKARA (Reuters) - The planned independence re...,worldnews,1
14236,Mnangagwa will be sworn in as Zimbabwe preside...,HARARE (Reuters) - Zimbabwe s former vice pres...,worldnews,1
8619,WATCH: The Media Gives MI’s Gov. Rick Snyder ...,Michigan s governor Rick Snyder is desperately...,News,0
18147,WHERE WAS MEDIA OUTRAGE After This WOMAN DELIB...,The media has been giving wall-to-wall coverag...,left-news,0


time: 8.26 ms (started: 2023-07-26 11:42:45 +00:00)


In [12]:
%cd ../FND/Fake-News-Detection

/content/drive/MyDrive/FND/Fake-News-Detection
time: 4.08 ms (started: 2023-07-26 11:43:02 +00:00)


In [13]:
import swifter
from preprocess import preprocess_text, vectorize_text, remove_nulls

print(data.shape)
data.reset_index(drop=True, inplace = True)
data['cleaned_text'] = data['text'].swifter.apply(lambda x : preprocess_text(x))
data['cleaned_title'] = data['title'].swifter.apply(lambda x : preprocess_text(x))

(44898, 4)


Pandas Apply:   0%|          | 0/44898 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/44898 [00:00<?, ?it/s]

time: 5min 22s (started: 2023-07-26 11:43:38 +00:00)


In [14]:
data = data.dropna()
data.shape

(44898, 6)

time: 84.7 ms (started: 2023-07-26 11:49:01 +00:00)


In [15]:
all_tokens = [text for text in data['cleaned_text']] + [title for title in data['cleaned_title']]

time: 24.6 ms (started: 2023-07-26 11:49:01 +00:00)


In [16]:
from gensim.models import Word2Vec

model = Word2Vec(all_tokens, vector_size=100, window=5, min_count=1, workers=4)
model.save("model_files/word2vec.model")

time: 1min 8s (started: 2023-07-26 11:49:01 +00:00)


In [17]:
data['text_vector'] = data['cleaned_text'].swifter.apply(lambda x : vectorize_text(text = x , model = model))
data['title_vector'] = data['cleaned_title'].swifter.apply(lambda x : vectorize_text(text = x , model = model))

Pandas Apply:   0%|          | 0/44898 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/44898 [00:00<?, ?it/s]

time: 26.8 s (started: 2023-07-26 11:50:09 +00:00)


In [21]:
print(data.shape)
data.head()

(44898, 8)


,title,text,subject,label,cleaned_text,cleaned_title,text_vector,title_vector
0,GA TOWN MANDATES GUN OWNERSHIP: Here’s What Ha...,The results of this town s gun ownership manda...,left-news,0,"[result, town, gun, ownership, mandat, show, h...","[ga, town, mandat, gun, ownership, happen, cri...","[0.5795905, -0.60971826, 0.5540126, -0.314273,...","[0.94410366, -0.6194935, 1.5652497, 0.3349229,..."
1,Turkey wants Iraq's Kurdish region to drop ref...,ANKARA (Reuters) - The planned independence re...,worldnews,1,"[ankara, reuter, plan, independ, referendum, k...","[turkey, want, iraq, kurdish, region, drop, re...","[-0.9729672, -0.86793333, 0.15772809, 1.239865...","[-1.3553503, -0.9331661, 1.9101762, 1.8061581,..."
2,Mnangagwa will be sworn in as Zimbabwe preside...,HARARE (Reuters) - Zimbabwe s former vice pres...,worldnews,1,"[harar, reuter, zimbabw, former, vice, presid,...","[mnangagwa, sworn, zimbabw, presid, zanu, pf]","[-0.8990433, -0.28727546, -1.080508, -0.251056...","[-1.9205784, -0.039547194, -0.9053559, -0.2041..."
3,WATCH: The Media Gives MI’s Gov. Rick Snyder ...,Michigan s governor Rick Snyder is desperately...,News,0,"[michigan, governor, rick, snyder, desper, tri...","[watch, medium, give, mi, gov, rick, snyder, t...","[0.2315432, -0.28155428, -0.040197365, -0.0807...","[-0.20861875, 0.75762075, -0.39959767, -0.4962..."
4,WHERE WAS MEDIA OUTRAGE After This WOMAN DELIB...,The media has been giving wall-to-wall coverag...,left-news,0,"[medium, give, wall, wall, coverag, horrif, vi...","[medium, outrag, woman, deliber, plow, car, la...","[0.70307994, -0.031985044, -0.1285875, -0.2149...","[1.2088757, 0.54285705, 0.2344868, -0.00762048..."


time: 31 ms (started: 2023-07-26 11:51:34 +00:00)


In [19]:
data.to_csv(r'cleaned-dataset/Final_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          44898 non-null  object
 1   text           44898 non-null  object
 2   subject        44898 non-null  object
 3   label          44898 non-null  int64 
 4   cleaned_text   44898 non-null  object
 5   cleaned_title  44898 non-null  object
 6   text_vector    44266 non-null  object
 7   title_vector   44898 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.7+ MB
time: 55.4 s (started: 2023-07-26 11:50:36 +00:00)


In [20]:
cleaned_data = remove_nulls(data)
cleaned_data.reset_index(inplace = True, drop = True)
cleaned_data.shape

(44266, 8)

time: 2.91 s (started: 2023-07-26 11:51:31 +00:00)


In [22]:
cleaned_data['title_vector'][0]

array([ 0.94410366, -0.6194935 ,  1.5652497 ,  0.3349229 , -0.8386245 ,
       -0.04496725,  0.31205326, -0.66732347, -0.13655774,  0.23479128,
       -0.8212076 , -0.10682598, -0.4896168 , -0.8914591 , -0.8206202 ,
       -0.09767123, -0.09809455,  0.28991783, -0.1978685 ,  0.8412615 ,
       -0.6308756 ,  0.69355005, -0.05576461, -0.11624432,  1.2728132 ,
       -1.6283174 ,  0.8307683 ,  0.22552195,  0.62416214, -0.24987802,
       -0.79905486,  0.65566117, -0.3898318 ,  1.752691  , -0.5432936 ,
        0.4065278 ,  0.21045388,  0.5078366 ,  0.4373182 , -1.3579217 ,
       -0.3041096 ,  0.81409705,  0.04337364,  0.41132498,  1.4714901 ,
        0.5381912 ,  0.29160094, -0.00583124, -0.16643563, -0.6506417 ,
       -0.6334894 , -0.06111388,  0.6861471 , -0.45886824, -0.7550589 ,
       -1.4670844 , -0.59744895,  0.3683265 ,  0.45357984,  0.03801733,
        1.4804574 ,  0.90456414,  0.3342724 , -0.7977038 , -1.6760168 ,
       -0.21265665, -0.24493629,  0.4409625 ,  0.6368282 , -1.22

time: 4.83 ms (started: 2023-07-26 11:51:34 +00:00)


In [23]:
np.vstack(cleaned_data['title_vector'])[0] == cleaned_data['title_vector'][0]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

time: 73.9 ms (started: 2023-07-26 11:51:34 +00:00)


In [24]:
vectors = np.vstack(cleaned_data['title_vector'])
vectors = np.float32(vectors)
labels = np.asarray(cleaned_data['label'])
labels = np.float32(labels)

time: 58.5 ms (started: 2023-07-26 11:51:34 +00:00)


In [25]:
cleanedup_npy = np.hstack((vectors, labels.reshape(-1, 1)))
cleanedup_npy = np.float32(cleanedup_npy)
np.save("cleaned-dataset/data_npy.npy", cleanedup_npy)

time: 882 ms (started: 2023-07-26 11:51:34 +00:00)
